In [153]:
from transformers import AutoModel
from tqdm import tqdm
import numpy as np
import torch
import json
import os

In [160]:
models_default = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_results", "test_results", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_results","test_results", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_results","test_results", "ELECTRA$_{G\:base}$"),
         ]
    )
]

models_wordsim = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_results_word_similarity","test_results_word_similarity", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_results_word_similarity","test_results_word_similarity", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_results_word_similarity","test_results_word_similarity", "ELECTRA$_{G\:base}$"),
         ]
    )
]


models_mlm = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_results__bert-base-cased_model_epoch_9_mlm", "test_results__bert-base-cased_model_epoch_9_mlm", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_results__google-electra-base-discriminator_model_epoch_9_mlm", "test_results__google-electra-base-discriminator_model_epoch_9_mlm", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_results__google-electra-base-generator_model_epoch_9_mlm", "test_results__google-electra-base-generator_model_epoch_9_mlm", "ELECTRA$_{G\:base}$ "),
         ]
    )
]


In [183]:
def generate_rows(models):
    for x in models:
        m = x[0]
        pool = x[1]
        c = x[2]
        json_names_labels = x[3]
        for p in pool:
            for (val_name, name, label) in json_names_labels:
                spearman, pearson = [], []
                std_spearman, std_pearson = [], []
                
                val_spearman, val_pearson = [], []
                for i in range(c):
                    res = json.load(open(f"../output/{m}/{p}/{i}_to_{i+1}/{name}.json"))
                    
                    std_1 = np.array(res["stdev_cosine_spearman_test"]) * 100
                    mean_1 = res["mean_cosine_spearman_test"] * 100
                    std_spearman.append((mean_1-std_1, mean_1+std_1))
                    spearman.append(mean_1)

                    std_2 = np.array(res["stdev_cosine_pearson_test"]) * 100
                    mean_2 = res["mean_cosine_pearson_test"] * 100
                    std_pearson.append((mean_2-std_2, mean_2+std_2))
                    pearson.append(mean_2)
                    
                    res_val = json.load(open(f"../output/{m}/{p}/{i}_to_{i+1}/{val_name}.json"))
                    val_spearman.append(res_val["mean_cosine_spearman_val"] * 100)
                    val_pearson.append(res_val["mean_cosine_pearson_val"] * 100)
                    
                argmax = np.argmax(val_spearman)
                print("{} & {} & {:.2f}/{:.2f} & {:.2f}/{:.2f} \\\\"
                    .format(
                        "}}".join("{\\text{".join(label.split("{")).split("}")), 
                        argmax, 
                        val_spearman[argmax], 
                        val_pearson[argmax], 
                        spearman[argmax], 
                        pearson[argmax]
                    )
                )
                test_s.append(spearman[argmax])
                test_p.append(pearson[argmax])
                val_s.append(val_spearman[argmax])
                val_p.append(val_pearson[argmax])
                
best = ""
                
test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_default):
    generate_rows(m)
best += "\n{} {} {} {} \n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))
print("\\midrule")


test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_wordsim):
    generate_rows(m)    
best += "{} {} {} {} \n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))
print("\\midrule")

test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_mlm):
    generate_rows(m)    
best += "{} {} {} {}\n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))
print(best)


BERT$_{\text{base}}$ & 12 & 86.04/85.94 & 82.76/83.01 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 82.14/82.18 & 75.29/76.96 \\
ELECTRA$_{\text{G\:base}}$ & 12 & 86.60/86.36 & 82.54/82.48 \\
\midrule
BERT$_{\text{base}}$ & 12 & 85.74/85.74 & 83.92/84.11 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 82.59/82.45 & 76.87/77.40 \\
ELECTRA$_{\text{G\:base}}$ & 12 & 86.47/86.20 & 82.58/82.69 \\
\midrule
BERT$_{\text{base}}$ & 12 & 85.89/85.67 & 82.70/82.64 \\
ELECTRA$_{\text{D\:base}}$ & 7 & 84.04/83.72 & 79.93/79.92 \\
ELECTRA$_{\text{G\:base}}$  & 11 & 85.55/85.21 & 80.99/80.91 \\

0 0 2 2 
0 0 2 2 
0 0 0 0



In [151]:
models = [
    [
        ("google-bert_uncased_L-2_H-128_A-2", 3,"BERT$_{tiny}$"),
        ("google-bert_uncased_L-4_H-256_A-4", 5,"BERT$_{mini}$"),
        ("google-bert_uncased_L-4_H-512_A-8", 5,"BERT$_{small}$"),
        ("google-bert_uncased_L-8_H-512_A-8", 9, "BERT$_{medium}$"),
        ("bert-base-cased", 13, "BERT$_{base}$"),
        ("bert-large-cased", 25, "BERT$_{large}$"),
    ],
    [
        ("google-electra-small-discriminator", 13, "ELECTRA$_{D\:small}$"),
        ("google-electra-base-discriminator", 13, "ELECTRA$_{D\:base}$"),
        ("google-electra-large-discriminator", 25, "ELECTRA$_{D\:large}$"),
    ],
    [
        ("google-electra-small-generator", 13, "ELECTRA$_{G\:small}$"),
        ("google-electra-base-generator", 13, "ELECTRA$_{G\:base}$"),
        ("google-electra-large-generator", 25, "ELECTRA$_{G\:large}$"),

    ]
]


def count_params(model, i):
    x = torch.tensor(0.)
    for p in model.embeddings.parameters():
        x += torch.prod(torch.tensor(p.shape))
    if i > 0:
        for p in model.encoder.layer[:i].parameters():
            x += torch.prod(torch.tensor(p.shape))
    return x/10**6

all_ = []
all_scores_params_names = []
for f in models:
   
    for m in tqdm(f):
        val_means, means = [], []
        val_pearson_means, pearson_means = [], []
        
        name = m[0] if "google" not in m[0] else m[0].replace("google-", "google/")
        model = AutoModel.from_pretrained(name)
        for i in range(m[1]):
            params = count_params(model, i)
            
            res = json.load(open(f"../output/{m[0]}/mean/{i}_to_{i+1}/test_results.json"))
            means.append([res["mean_cosine_spearman_test"]*100, params])
            pearson_means.append(res["mean_cosine_pearson_test"]*100)
            
            res = json.load(open(f"../output/{m[0]}/mean/{i}_to_{i+1}/val_results.json"))
            val_means.append(res["mean_cosine_spearman_val"] * 100)
            val_pearson_means.append(res["mean_cosine_pearson_val"] * 100)

        tmp_means = means[:]
        argmax = np.argmax(val_means)
        s, num_param = np.array(means)[argmax]
        p = pearson_means[argmax]
        
        val_s = val_means[argmax]
        val_p = val_pearson_means[argmax]
        
        all_scores_params_names.append([s, num_param, val_s, p, val_p, argmax, m[2]])
        if s != tmp_means[-1][0]:
            all_scores_params_names.append(
                [
                    tmp_means[-1][0], 
                    tmp_means[-1][1],
                    val_means[-1],
                    pearson_means[-1],
                    val_pearson_means[-1],
                    argmax,
                    "\:last}".join(m[2].split("}"))]
            )
        
argm_score = np.argmax([p[0] for p in all_scores_params_names])
argm_param = np.argmin([p[1] for p in all_scores_params_names])
argm_val_s = np.argmax([p[2] for p in all_scores_params_names])
argm_p = np.argmax([p[3] for p in all_scores_params_names])
argm_val_p = np.argmax([p[4] for p in all_scores_params_names])

print(
    " max spearman : ", all_scores_params_names[argm_score][-1], "\n",
    "min params: ", all_scores_params_names[argm_param][-1], "\n",
    "max val spearman: ", all_scores_params_names[argm_val_s][-1],"\n", 
    "pearson at max val spearman: ", all_scores_params_names[argm_p][-1],"\n", 
    "pearson val at max val spearman: ", all_scores_params_names[argm_val_p][-1],"\n", 
    "\n"
)

for p in all_scores_params_names:
    rounded = np.round(np.array(p[:-1]).astype(np.float32), decimals=2)
    #print(rounded)
    print(
        "}}".join("{\\text{".join(p[-1].split("{")).split("}")) + " & " +
        str(int(rounded[5])) + " & " +
        "{:.2f} & ".format(rounded[1]) +
        "{:.2f}/{:.2f} & ".format(rounded[2], rounded[4]) +
        "{:.2f}/{:.2f} ".format(rounded[0], rounded[3]) +
        "\\\\"
    )

        

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.43it/s]

 max spearman :  BERT$_{large}$ 
 min params:  BERT$_{tiny}$ 
 max val spearman:  BERT$_{large}$ 
 pearson at max val spearman:  BERT$_{large}$ 
 pearson val at max val spearman:  BERT$_{large}$ 
 

BERT$_{\text{tiny}}$ & 2 & 4.37 & 78.23/77.58 & 69.71/70.57 \\
BERT$_{\text{mini}}$ & 4 & 11.10 & 83.10/82.47 & 75.64/76.36 \\
BERT$_{\text{small}}$ & 4 & 28.50 & 85.12/84.92 & 79.32/79.68 \\
BERT$_{\text{medium}}$ & 8 & 41.11 & 85.72/85.39 & 80.84/81.10 \\
BERT$_{\text{base}}$ & 12 & 107.72 & 86.04/85.94 & 82.76/83.01 \\
BERT$_{\text{large}}$ & 24 & 332.53 & 88.27/88.25 & 85.58/85.75 \\
ELECTRA$_{\text{D\:small}}$ & 1 & 4.76 & 79.72/79.26 & 68.82/69.63 \\
ELECTRA$_{\text{D\:small\:last}}$ & 1 & 13.45 & 74.02/73.15 & 66.54/67.00 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 45.10 & 82.14/82.18 & 75.29/76.96 \\
ELECTRA$_{\text{D\:base\:last}}$ & 3 & 108.89 & 72.32/71.56 & 66.93/67.35 \\
ELECTRA$_{\text{D\:large}}$ & 12 & 182.94 & 84.36/84.40 & 80.54/80.71 \\
ELECTRA$_{\text{D\:large\:last}}$ & 12 & 33